# Instalación de la librerías necesarias

In [1]:
!pip install langchain==0.1.20 -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.8 MB/s eta 0:00:00


In [2]:
!pip install pypdf -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.3 MB/s eta 0:00:00


# Carga de un documento pdf

In [3]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/Anexo Curso_ Desarrollo de Aplicaciones LLM con LangChain, LlamaIndex y OpenAI.pdf")
documento = loader.load()

In [4]:
print(len(documento))

5


# Dividimos el pdf en partes

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=400
)

In [7]:
document_chunks = text_splitter.split_documents(documento)

In [8]:
print(f"Ahora tenemos {len(document_chunks)} partes.")

Ahora tenemos 5 partes.


# Inicializamos el modelo de embeddings que vamos a utilizar para convertir las partes de texto en vectores numéricos (embeddings)

In [9]:
!pip install langchain-openai==0.1.3 -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.0/326.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [10]:
from langchain_openai import OpenAIEmbeddings

In [11]:
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

In [12]:
embeddings_model = OpenAIEmbeddings(api_key=openai_api_key)

# Cargamos los embeddings en una base de datos vectorial

In [13]:
!pip install faiss-cpu -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 47.5 MB/s eta 0:00:00


In [14]:
from langchain.vectorstores import FAISS

In [15]:
stored_embeddings = FAISS.from_documents(document_chunks, embeddings_model)

# Creamos una Cadena de Preguntas y Respuetas con Recuperación

In [16]:
from langchain_openai import OpenAI

In [17]:
llm = OpenAI(api_key=openai_api_key)

In [18]:
from langchain.chains import RetrievalQA

In [19]:
QA_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever()
)

# Probamos nuestra aplicación de Preguntas y Respuestas relacionadas al documento de interés

In [20]:
question = """
De qué trata el documento
"""

In [21]:
QA_chain.invoke(question)

{'query': '\nDe qué trata el documento\n',
 'result': ' El documento trata sobre un curso de Desarrollo de Aplicaciones LLM con LangChain, LlamaIndex y OpenAI.'}

In [22]:
question2 = """
A quién va dirigido el curso?
"""

In [23]:
QA_chain.invoke(question2)

{'query': '\nA quién va dirigido el curso?\n',
 'result': ' \nEl curso está dirigido a docentes y alumnos del Departamento de Informática, agentes del Ministerio de Ciencia e Innovación Tecnológica de Catamarca, agentes de la Dirección General de Modernización de la Municipalidad de San Fernando del Valle de Catamarca, agentes del Nodo Tecnológico de la Municipalidad de San Fernando del Valle de Catamarca, agentes de la Secretaría de Modernización del Gobierno de la Provincia de Catamarca, y al público en general con experiencia en la temática del curso. '}

In [24]:
question3 = """
Cuánto cuesta el curso?
"""

In [25]:
QA_chain.invoke(question3)

{'query': '\nCuánto cuesta el curso?\n',
 'result': ' \nEl curso es completamente gratuito para todos los participantes.'}